## 5.6 深度卷积神经网络（AlexNet）

* LeNet 在较小数据集上可以取得较好成绩，但是在更大真实数据集上表现并不尽人意。


In [9]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from livelossplot.tf_keras import PlotLossesCallback
from  skimage import transform
import numpy as np

In [10]:
model = keras.Sequential([
    keras.layers.Lambda(lambda img: tf.image.resize(img, (224, 224)), input_shape=(28, 28, 1)),
    keras.layers.Conv2D(96, (11, 11), strides=4, activation='relu', input_shape=(224, 224, 1)),
    keras.layers.MaxPool2D(3, strides=2),
    keras.layers.Conv2D(256, (5, 5), padding='same', activation='relu'),
    keras.layers.MaxPool2D(3, strides=2),
    keras.layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
    keras.layers.Conv2D(384, (3, 3), padding='same', activation='relu'),
    keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
    keras.layers.MaxPool2D(3, strides=2),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 224, 224, 1)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 54, 54, 96)        11712     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 26, 26, 96)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 26, 26, 256)       614656    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 12, 12, 384)       885120    
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 12, 12, 384)      

In [11]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train = x_train/255.
x_test = x_test/255.
#x_train = np.asarray([transform.resize(image, (224, 224, 1)) for image in x_train])

In [ ]:
def metric_accuracy(y_true, y_pred):
    ytrue = K.flatten(y_true)
    ypred = K.cast(K.argmax(y_pred, axis=-1), K.floatx())
    acc = K.equal(ytrue, ypred)
    return K.mean(acc)

model.compile(optimizer='adam', 
              loss=keras.losses.sparse_categorical_crossentropy,
              metrics=[metric_accuracy])

model.fit(x_train.reshape(x_train.shape[0], 28, 28, 1), y_train, epochs=5, batch_size=128,
          validation_data=(x_test.reshape(x_test.shape[0], 28, 28, 1), y_test),
          callbacks=[PlotLossesCallback()])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
15744/60000 [======>.......................] - ETA: 20:17 - loss: 0.9728 - metric_accuracy: 0.6300